In [1]:
source("../tools/formating/formating.R")

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    

In [2]:
path <- "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"

In [3]:
result <- GetMetadataFromSeurat(path)
result

Updating from v2.X to v3.X

Validating object structure

Updating object slots

Ensuring keys are in the proper structure

Ensuring feature names don't have underscores or pipes

Object representation is consistent with the most current Seurat version



,orig.ident,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,free_annotation,cell_ontology_class,res.0.4,previous_free_annotation,previous_cell_ontology_class,cluster.ids,cell_ontology_id,nCount_RNA,nFeature_RNA
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
10X_P4_3_AAAGTAGAGATGCCAG,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.21960937,Bladder mesenchymal cell,bladder cell,0,NA,NA,0,CL:1001319,10291,2853
10X_P4_3_AACCGCGTCCAACCAA,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.17671233,Bladder mesenchymal cell,bladder cell,2,NA,NA,2,CL:1001319,17520,4194
10X_P4_3_AACTCCCGTCGGGTCT,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.19244438,Bladder mesenchymal cell,bladder cell,5,NA,NA,5,CL:1001319,12944,3347
10X_P4_3_AACTCTTAGTTGCAGG,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.35638461,Luminal bladder epithelial cell,bladder urothelial cell,3,NA,NA,3,CL:1001428,19594,3851
10X_P4_3_AACTCTTTCATAACCG,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.15711936,Bladder mesenchymal cell,bladder cell,2,NA,NA,2,CL:1001319,14594,3809
10X_P4_3_AAGACCTAGATCCGAG,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.23778009,Monocyte/Macrophage,leukocyte,6,NA,NA,6,CL:0000738,19190,3608
10X_P4_3_AAGACCTAGGTGGGTT,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.32809182,Basal bladder epithelial cell,bladder urothelial cell,1,NA,NA,1,CL:1001428,24484,4347
10X_P4_3_AAGGAGCGTGCAACTT,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.19020979,Bladder mesenchymal cell,bladder cell,0,NA,NA,0,CL:1001319,5005,1960
10X_P4_3_AAGTCTGAGATAGTCA,10X,10X_P4_3,Bladder,,M,3-M-8,0,0.22597153,Luminal bladder epithelial cell,bladder urothelial cell,4,NA,NA,4,CL:1001428,6253,2236


In [ ]:
suffix <- tolower(get_suffix(path))
suffix

In [ ]:
seurat_object <- LoadSeurat(path)
seurat_object

In [ ]:
'umap' %in% names(seurat_object@reductions)

In [ ]:
genes <- rownames(seurat_object)
typeof(genes)
genes

In [ ]:
cells <- Cells(seurat_object)
typeof(cells)
cells

In [ ]:
pca <- Embeddings(object = seurat_object, reduction = "pca")
typeof(pca)
pca

In [ ]:
name <- load(path)
seurat_object <- UpdateSeuratObject(get(name))

In [ ]:
seurat_object

In [ ]:
features <- seurat_object[['RNA']]@var.features
typeof(features)
features

In [ ]:
#获取当前用的Idents
Idents(seurat_object)
levels(seurat_object)

In [ ]:
#总数目
nCells <- ncol(seurat_object)
nGenes <- nrow(seurat_object)

nCells
nGenes